# IVision
   Intelligent Attendace System

In [2]:
#from Videoprocessing import *
from Classifier import *
#import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage import data, color, exposure
from sklearn.svm import LinearSVC
import argparse
import cv2
import os.path
import numpy as np
import webbrowser  

In [2]:
#vid=Videoprocess()
#vid.Extractfaces()


#Video Capture For Tagging+Training

In [34]:
#This captures frames from the given video and individually detects 
#and stores faces from those frames to the "faces" directory
cap = cv2.VideoCapture('IMG_4100.MOV')
face_cascade = cv2.CascadeClassifier('C:\opencv\sources\data\haarcascades\haarcascade_frontalface_default.xml')
print cap.grab()
i=0
j=0
ret, frame = cap.read()
while(ret):
    # Capture frame-by-frame
    #print j
    ret, frame = cap.read()
        # Our operations on the frame come here
    if j%10==0:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.2, )
        for (x,y,w,h) in faces:
            cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]
            cv2.imwrite('faces/Testimg'+str(i)+'.png',frame[y:y+h,x:x+w])
            i=i+1
        # Display the resulting frame
        cv2.imshow('frame',frame)
        j+=1
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        j+=1

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

True


In [35]:
#converts all the faces to a standard size of 50x50
#overwrites in the "faces" directory
num_files = sum(os.path.isfile(os.path.join('faces', f)) for f in os.listdir('faces'))
print num_files
t=[]
for i in range(0,num_files-1):
    img=cv2.imread('faces/Testimg' +str(i)+'.png',0)
    simg3=cv2.resize(img,(50,50))
    t.append(simg3.reshape(2500))
    #simg3=cv2.resize(img,(100,100))
    cv2.imwrite('faces/Testimg'+str(i)+'.png',simg3)    
print len(t)   
data=np.array(t)
print data.shape

131
130
(130L, 2500L)


In [36]:
num_files = sum(os.path.isfile(os.path.join('faces', f)) for f in os.listdir('faces'))
print num_files

131


In [37]:
#Applies HOG on all faces
Train=[]
Test=[]
#hist=np.array(hist)
for i in range(0,num_files-1):
    image=cv2.imread('faces/Testimg' +str(i)+'.png',0)
    fd, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16),cells_per_block=(1, 1), visualise=True)
    hog_image=np.array(hog_image)
    hog_image=hog_image.ravel()
    Train.append(hog_image)
    Test.append(hog_image)
a=np.array(Train)
b=np.array(Test)
print a.shape
print b.shape


#for i in range(0,num_files-1):
#    image=cv2.imread('UniqueFaces/Testimg' +str(i)+'.png',0)
#    fd, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16),cells_per_block=(1, 1), visualise=True)
#    Test.append(hog_image[:])   

from scipy.spatial.distance import cdist
dists=cdist(Test,Train)
print dists[1]
if num_files>=175:
    k=12
else:
    k=8

(130L, 2500L)
(130L, 2500L)
[  50.3752064     0.           41.1193919    28.7832497    56.55263351
   12.0489158    43.89272096   31.40106975   56.67878323   56.99790389
   12.09283408   44.0579369    35.74567904   58.66778572   50.08937415
   11.39528209   41.4377925    33.3734993    51.57564308   18.31699192
   45.75014092   35.89032667   69.02419723   51.81013837   40.40430115
   51.82007217   18.90811215   48.76528075   32.23888297   70.85495762
   46.86977375   31.76763579   47.70053993   17.48990012   45.10315361
   69.47566955   48.39918728   55.87745669   48.75337913   25.35036901
   44.91870699   36.14779373   53.42412611   68.58149379   51.31196898
   63.89966186   63.59077611   55.34892298   35.97847768   47.58497111
   65.90758257   40.99816295   54.40752482   37.30682454   52.44397387
   69.71261412   49.06685621   45.87501136   37.43200476   43.58311144
   46.88114647   55.56294661   48.45652043   54.88464459   40.07642756
   53.86983746   68.37302342   37.37018621   50.4

In [38]:
#measures distances between all images 
#this is being done to reduce number of duplicate faces
num_test = dists.shape[0]
y_pred = np.zeros((num_test,4))
j=1
for j in xrange(num_test):
    dists[j][j]=1000
    for k in range(0,4):
        a=float(np.min(dists[j]))
        b=float(np.argmin(dists[j]))
        #print a,b
        y_pred[j][k]=b;
        dists[j][b]=1000;

print y_pred

[[  23.   14.   18.   30.]
 [  15.    5.   10.   33.]
 [  11.   16.    6.   20.]
 [   7.   17.  102.   87.]
 [   9.   18.   14.   57.]
 [  10.    1.   15.   19.]
 [  16.    2.   20.   11.]
 [  17.   12.   28.    3.]
 [  13.   22.   84.   29.]
 [   4.   14.   18.   57.]
 [   5.   15.    1.   26.]
 [   2.   20.    6.   16.]
 [  17.    7.   28.   21.]
 [  84.    8.   22.   29.]
 [  30.   60.   57.    0.]
 [  10.    1.   19.    5.]
 [   6.    2.   20.   11.]
 [  12.    7.   28.   77.]
 [  23.   14.   57.    0.]
 [  15.   26.   10.    1.]
 [  11.   16.    6.    2.]
 [  28.   17.   87.  102.]
 [  29.   35.   84.   13.]
 [  18.    0.   57.   14.]
 [  58.   83.   92.  103.]
 [  97.   80.  120.   70.]
 [  19.   15.   10.    1.]
 [  20.   76.   32.   11.]
 [  77.   17.   31.    7.]
 [  22.   35.   84.   43.]
 [  14.   57.   60.   36.]
 [  77.   28.   87.   41.]
 [  74.   42.   56.   27.]
 [   1.   15.   39.   19.]
 [  97.   38.   89.   80.]
 [  43.   66.   75.   29.]
 [  57.   30.   60.   23.]
 

In [39]:
#stores the resulting data set in "images" directory to be tagged by user
#using web interface
#These tagged faces then used to train classifier
#Note: First iteration of algorithm stores images in "images2" dir
extras=[]
done=[]
count=0
counter=0
for i in range(0,len(y_pred)):
    for j in y_pred[i]:
        if ((count==0) and (j not in extras) and (j not in done)):
            img=cv2.imread('faces/Testimg' +str(i)+'.png',0)
            done.append(int(j))
            #print j
            cv2.imwrite('images2/Testimg'+str(counter)+'.png',img) 
            counter+=1
            count=1
            for m in y_pred[i]:
                if m != j:
                    extras.append(int(m))
    count=0
#print extras

In [11]:
num_files = sum(os.path.isfile(os.path.join('images2', f)) for f in os.listdir('images2'))
print num_files
t=[]
for i in range(0,num_files-1):
    img=cv2.imread('images2/Testimg' +str(i)+'.png',0)
    simg3=cv2.resize(img,(50,50))
    t.append(img.reshape(2500))
    simg3=cv2.resize(img,(100,100))
    cv2.imwrite('images2/Testimg'+str(i)+'.png',simg3)    
print len(t)   
data=np.array(t)
print data.shape
        
fdata=np.float32(data)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 1.0)
ret,label,center=cv2.kmeans(fdata,34,None,criteria,20,cv2.KMEANS_RANDOM_CENTERS)
for i in range(0,34):
    print len(center),type(center)
    print len(label),type(label)
    print center[0].shape
    imgc=np.uint8(center[i].reshape(50,50))
    print type(ret),ret
    cv2.imwrite('images/Testimg'+str(i+10)+'.png',center[i].reshape(50,50))
#Stored unique faces in "images" dir

64


ValueError: total size of new array must be unchanged

In [41]:
url="localhost"
webbrowser.get("C:/Program Files (x86)/Google/Chrome/Application/chrome.exe %s").open(url)

True

In [19]:
#import pdb
#%pdb on
classifier1=Classify()
classifier1.trainsystem()
#the classifier is being trained here

106


#Video Capture For TestData

In [3]:

cap = cv2.VideoCapture('IMG_4100.MOV')
face_cascade = cv2.CascadeClassifier('C:\opencv\sources\data\haarcascades\haarcascade_frontalface_default.xml')
print cap.grab()
i=0
j=0
ret, frame = cap.read()
while(ret):
    # Capture frame-by-frame
    #print j
    ret, frame = cap.read()
        # Our operations on the frame come here
    if j%10==0:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.2, )
        for (x,y,w,h) in faces:
            cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]
            cv2.imwrite('faces/Testimg'+str(i)+'.png',frame[y:y+h,x:x+w])
            i=i+1
        # Display the resulting frame
        cv2.imshow('frame',frame)
        j+=1
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        j+=1

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

True


In [4]:
num_files = sum(os.path.isfile(os.path.join('faces', f)) for f in os.listdir('faces'))
print num_files
t=[]
for i in range(0,num_files-1):
    img=cv2.imread('faces/Testimg' +str(i)+'.png',0)
    simg3=cv2.resize(img,(50,50))
    t.append(simg3.reshape(2500))
    #simg3=cv2.resize(img,(100,100))
    cv2.imwrite('faces/Testimg'+str(i)+'.png',simg3)    
print len(t)   
data=np.array(t)
print data.shape

131
130
(130L, 2500L)


In [14]:
num_files = sum(os.path.isfile(os.path.join('faces', f)) for f in os.listdir('faces'))
print num_files

131


In [15]:
#Applies HOG on all faces
Train=[]
Test=[]
#hist=np.array(hist)
for i in range(0,num_files-1):
    image=cv2.imread('faces/Testimg' +str(i)+'.png',0)
    fd, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16),cells_per_block=(1, 1), visualise=True)
    hog_image=np.array(hog_image)
    hog_image=hog_image.ravel()
    Train.append(hog_image)
    Test.append(hog_image)
a=np.array(Train)
b=np.array(Test)
print a.shape
print b.shape


#for i in range(0,num_files-1):
#    image=cv2.imread('UniqueFaces/Testimg' +str(i)+'.png',0)
#    fd, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16),cells_per_block=(1, 1), visualise=True)
#    Test.append(hog_image[:])   

from scipy.spatial.distance import cdist
dists=cdist(Test,Train)
print dists[1]
if num_files>=175:
    k=12
else:
    k=8

(130L, 2500L)
(130L, 2500L)
[  50.3752064     0.           41.1193919    28.7832497    56.55263351
   12.0489158    43.89272096   31.40106975   56.67878323   56.99790389
   12.09283408   44.0579369    35.74567904   58.66778572   50.08937415
   11.39528209   41.4377925    33.3734993    51.57564308   18.31699192
   45.75014092   35.89032667   69.02419723   51.81013837   40.40430115
   18.90811215   51.82007217   48.76528075   32.23888297   70.85495762
   46.86977375   31.76763579   47.70053993   17.48990012   45.10315361
   69.47566955   48.39918728   55.87745669   48.75337913   25.35036901
   44.91870699   36.14779373   53.42412611   68.58149379   51.31196898
   63.89966186   63.59077611   55.34892298   35.97847768   47.58497111
   65.90758257   40.99816295   54.40752482   37.30682454   52.44397387
   69.71261412   49.06685621   45.87501136   37.43200476   43.58311144
   46.88114647   55.56294661   48.45652043   54.88464459   40.07642756
   53.86983746   68.37302342   37.37018621   50.4

In [16]:
#measures distances between all images 
#this is being done to reduce number of duplicate faces
num_test = dists.shape[0]
y_pred = np.zeros((num_test,4))
j=1
for j in xrange(num_test):
    dists[j][j]=1000
    for k in range(0,4):
        a=float(np.min(dists[j]))
        b=float(np.argmin(dists[j]))
        #print a,b
        y_pred[j][k]=b;
        dists[j][b]=1000;

print y_pred

[[  23.   14.   18.   30.]
 [  15.    5.   10.   33.]
 [  11.   16.    6.   20.]
 [   7.   17.  102.   87.]
 [   9.   18.   14.   57.]
 [  10.    1.   15.   19.]
 [  16.    2.   20.   11.]
 [  17.   12.   28.    3.]
 [  13.   22.   84.   29.]
 [   4.   14.   18.   57.]
 [   5.   15.    1.   25.]
 [   2.   20.    6.   16.]
 [  17.    7.   28.   21.]
 [  84.    8.   22.   29.]
 [  30.   60.   57.    0.]
 [  10.    1.   19.    5.]
 [   6.    2.   20.   11.]
 [  12.    7.   28.   77.]
 [  23.   14.   57.    0.]
 [  15.   25.   10.    1.]
 [  11.   16.    6.    2.]
 [  28.   17.   87.  102.]
 [  29.   35.   84.   13.]
 [  18.    0.   57.   14.]
 [  58.   83.   92.  103.]
 [  19.   15.   10.    1.]
 [  97.   80.  120.   70.]
 [  20.   76.   32.   11.]
 [  77.   17.   31.    7.]
 [  22.   35.   84.   43.]
 [  14.   57.   60.   36.]
 [  77.   28.   87.   41.]
 [  74.   42.   56.   27.]
 [   1.   15.   39.   19.]
 [  97.   38.   89.   80.]
 [  43.   66.   75.   29.]
 [  57.   30.   60.   23.]
 

In [17]:
#stores the resulting data set in "TestData" directory to used by our classifier
#For testing purposes
extras=[]
done=[]
count=0
counter=0
for i in range(0,len(y_pred)):
    for j in y_pred[i]:
        if ((count==0) and (j not in extras) and (j not in done)):
            img=cv2.imread('faces/Testimg' +str(i)+'.png',0)
            done.append(int(j))
            #print j
            cv2.imwrite('images2/Testimg'+str(counter)+'.png',img) 
            counter+=1
            count=1
            for m in y_pred[i]:
                if m != j:
                    extras.append(int(m))
    count=0
#print extras

In [18]:
num_files = sum(os.path.isfile(os.path.join('images2', f)) for f in os.listdir('images2'))
print num_files
t=[]
for i in range(0,num_files-1):
    img=cv2.imread('images2/Testimg' +str(i)+'.png',0)
    simg3=cv2.resize(img,(50,50))
    t.append(img.reshape(2500))
    simg3=cv2.resize(img,(100,100))
    cv2.imwrite('images2/Testimg'+str(i)+'.png',simg3)    
print len(t)   
data=np.array(t)
print data.shape
        
fdata=np.float32(data)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 1.0)
ret,label,center=cv2.kmeans(fdata,34,None,criteria,20,cv2.KMEANS_RANDOM_CENTERS)
for i in range(0,34):
    print len(center),type(center)
    print len(label),type(label)
    print center[0].shape
    imgc=np.uint8(center[i].reshape(50,50))
    print type(ret),ret
    cv2.imwrite('TestData/Testimg'+str(i)+'.png',center[i].reshape(50,50))
#Stored unique faces in "images" dir

64
63
(63L, 2500L)
34 <type 'numpy.ndarray'>
63 <type 'numpy.ndarray'>
(2500L,)
<type 'float'> 11003596.9297
34 <type 'numpy.ndarray'>
63 <type 'numpy.ndarray'>
(2500L,)
<type 'float'> 11003596.9297
34 <type 'numpy.ndarray'>
63 <type 'numpy.ndarray'>
(2500L,)
<type 'float'> 11003596.9297
34 <type 'numpy.ndarray'>
63 <type 'numpy.ndarray'>
(2500L,)
<type 'float'> 11003596.9297
34 <type 'numpy.ndarray'>
63 <type 'numpy.ndarray'>
(2500L,)
<type 'float'> 11003596.9297
34 <type 'numpy.ndarray'>
63 <type 'numpy.ndarray'>
(2500L,)
<type 'float'> 11003596.9297
34 <type 'numpy.ndarray'>
63 <type 'numpy.ndarray'>
(2500L,)
<type 'float'> 11003596.9297
34 <type 'numpy.ndarray'>
63 <type 'numpy.ndarray'>
(2500L,)
<type 'float'> 11003596.9297
34 <type 'numpy.ndarray'>
63 <type 'numpy.ndarray'>
(2500L,)
<type 'float'> 11003596.9297
34 <type 'numpy.ndarray'>
63 <type 'numpy.ndarray'>
(2500L,)
<type 'float'> 11003596.9297
34 <type 'numpy.ndarray'>
63 <type 'numpy.ndarray'>
(2500L,)
<type 'float'> 11003

In [20]:
classifier1.predict('TestData')
#Predicting attendance here

35
niazi

saad

niazi

sheri

sheri

sheri

usama

arslan

sheri

sufyan

ahsan

niazi

sheri

arslan

mohsin

sheri

saad

rajabilal

mohsin

mohsin

sheri

arslan

saad

sufyan

hamza

sheri

ahsan

saad

ali

sheri

saad

kahoot

arslan

